In [2]:
import numpy as np
import time

## Generate the Data

We first generate a random dataset with number of features (m = 10) and number of instances (n = 100)
We also generate a random label vector y \in {-1,1}

In [5]:
n = 100 # Number of instances
m = 10  # Number of Features 

X = np.random.rand(n,m)
y = np.random.rand(n)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)
print(y)
print(X)

[ 1  1 -1 -1 -1  1 -1  1  1  1 -1  1 -1  1 -1  1  1  1 -1 -1  1  1 -1 -1
 -1 -1 -1 -1  1 -1 -1  1 -1 -1  1  1  1 -1  1 -1 -1 -1  1  1  1 -1  1  1
  1 -1  1 -1  1 -1 -1 -1 -1  1 -1  1  1 -1  1  1 -1  1  1 -1 -1  1  1 -1
  1  1  1 -1  1  1  1 -1 -1  1  1 -1 -1  1  1 -1 -1 -1 -1  1 -1 -1 -1  1
  1 -1 -1 -1]
[[0.15081579 0.02060775 0.76881967 0.72560174 0.46650267 0.98982589
  0.91384915 0.81710788 0.29143393 0.55795121]
 [0.86394994 0.90318432 0.78147468 0.81914016 0.1059121  0.44348636
  0.74783135 0.17751439 0.95526117 0.60228485]
 [0.68263416 0.28028993 0.73856586 0.96458103 0.48799289 0.31165456
  0.08107802 0.89068509 0.36414014 0.01643942]
 [0.49802953 0.38370109 0.82619215 0.98252769 0.35166422 0.46767241
  0.34375141 0.11204718 0.57158609 0.63917095]
 [0.18790744 0.54965513 0.99397204 0.20613624 0.57725534 0.73939673
  0.16331439 0.65325905 0.34976847 0.4567399 ]
 [0.61553026 0.61245476 0.40400442 0.98580638 0.68637067 0.80192352
  0.7057647  0.96929697 0.45785726 0.05656022]
 [0.

## A Simple naive Implementation of the Logistic Loss 

Below is a simple naive implementation of Logistic Loss. We directly plug in the formula with a simple for loop!

In [6]:
def LogisticLossNaive(w, X, y, lam):
    # Computes the cost function for all the training samples
    f = 0
    g = 0
    for i in range(len(X)):
        featureweightProd = np.dot(X[i],w)
        f = f + np.log(1 + np.exp(-y[i]*featureweightProd))
        g = g + -y[i]/(1 + np.exp(y[i]*featureweightProd))*X[i]
    f = f + 0.5*lam*np.sum(w*w)
    g = g + lam*w.reshape(1,-1)
    return [f, g]     

In [7]:
start = time.time()
[f,g] = LogisticLossNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Time Taken = 0.0070149898529052734
Function value = [134.67014975]
Printing Gradient:
[[21.96211495 21.00490527 23.2016018  25.46196684 26.97171059 20.9702425
  19.03946367 20.76246686 22.74056883 21.53568644]]


This looks great! Can we ship this code? Well, we might be able to, but we should ideally test this out more! 

For one, the above scenario is very simplistic! In practice we have much larger number of features and instances. Let us increase the number of features m to 10000

In [7]:
n = 100
m = 10000

X = np.random.rand(n,m)
y = np.random.rand(n)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

start = time.time()
[f,g] = LogisticLossNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Time Taken = 0.010155200958251953
Function value = [inf]
Printing Gradient:
[[28.04187539 28.94383048 29.70031591 ... 31.13802037 27.57701899
  25.72323391]]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in exp
  


## Solving numerical issues with log-sum-exp!

We see that we have run into numerical issues! The main reason is the naive implementation does not have numerical stability in the definition of log-sum-exp. Once -y*x*w becomes large positive, exp(-y*x*w) will become Inf and Log(Inf) is Inf!

We can solve this by either defining our own function which protects against such numerical issues, or use the inbuilt function logaddexp. 

However it is super critical to be aware of numerical issues!

Lets define a function LogisticLossFor which is the same as above just fixing the numerical issue above!

In [11]:
def LogisticLossFor(w, X, y, lam):
    # Computes the cost function for all the training samples
    m = X.shape[0]
    f = 0
    g = 0
    for i in range(len(X)):
        featureweightProd = np.dot(X[i],w)
        f = f + np.logaddexp(0, -y[i]*featureweightProd)
        g = g + -y[i]/(1 + np.exp(y[i]*featureweightProd))*X[i]
    f = f + 0.5*lam*np.sum(w*w)
    g = g + lam*w.reshape(1,-1)
    return [f, g]     

In [9]:
start = time.time()
[f,g] = LogisticLossFor(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Time Taken = 0.00616002082824707
Function value = [144628.44285299]
Printing Gradient:
[[28.04187539 28.94383048 29.70031591 ... 31.13802037 27.57701899
  25.72323391]]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


## Lets make sure the two definitions are the same!

The above clearly fixed the Inf issue! However every time we write a new code, we should ensure our code is correct and the best way to do it is by checking with a previous working version!

In [11]:
import numpy as np
n = 100
m = 10

X = np.random.rand(n,m)
y = np.random.rand(n)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

start = time.time()
[f1,g1] = LogisticLossNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value Naive = " + str(f1))
print("Printing Gradient Naive:")
print(g1)

start = time.time()
[f2,g2] = LogisticLossFor(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value For = " + str(f2))
print("Printing Gradient For:")
print(g2)

Time Taken = 0.0017960071563720703
Function value Naive = [146.9464845]
Printing Gradient Naive:
[[22.21932155 21.94511061 25.01754849 23.57480241 24.7308955  26.42184361
  22.82138964 23.90039954 23.95644553 24.0426967 ]]
Time Taken = 0.002031087875366211
Function value For = [146.9464845]
Printing Gradient For:
[[22.21932155 21.94511061 25.01754849 23.57480241 24.7308955  26.42184361
  22.82138964 23.90039954 23.95644553 24.0426967 ]]


## For Loop in Python == Slow Code

Great, we have fixed the Inf issue now! But while this code might be correct, is this going to be fast? We have a For loop in python which is clearly an issue!

First let us see how slow the code is! Let us increase n to 1000000 and m to 1000, which are somewhat more realistic (though still far from real world).

In [10]:
n = 1000000
m = 100

X = np.random.rand(n,m)
y = np.random.rand(n)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

start = time.time()
[f,g] = LogisticLossFor(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))

Time Taken = 13.130612850189209
Function value = [12994789.70183541]


## Speeding up the code!

With m = 100, it takes around 10 seconds and with m = 1000, it is already 65 seconds (you can try it at home). 
With each 10x increase in m or n, the time taken increases exponentially!

Lets now vectorize the code below.

In [9]:
def LogisticLoss(w, X, y, lam):
    # Computes the cost function for all the training samples
    m = X.shape[0]
    Xw = np.matmul(X,w)
    yT = y.reshape(-1,1)
    yXw = np.multiply(yT,Xw)
    f = np.sum(np.logaddexp(0,-yXw)) + 0.5*lam*np.sum(w*w)
    gMul = np.exp(-yXw)/(1 + np.exp(-yXw))
    ymul = -1*yT*gMul
    g =  np.matmul(ymul.reshape(1,-1),X) + lam*w.reshape(1,-1)
    #g = np.dot(X.T,ymul) + lam*w.reshape(1,-1)
    g = g.reshape(-1,1)
    return [f, g]

In [12]:
n = 100 # Number of instances
m = 10  # Number of Features 

X = np.random.rand(n,m)
y = np.random.rand(n)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

[f1,g1] = LogisticLoss(w,X,y,1)
print("Function value = " + str(f1))

[f2,g2] = LogisticLossFor(w,X,y,1)
print("Function value = " + str(f2))


Function value = 133.24380504169818
Function value = [133.24380504]


## Checking gradient implementations!

So far so good! But how do we verify if our gradient implementation is correct?
We can test out our loss function analytically, but what if we make a mistake in computing the gradient? We can numerically compute the gradient to ensure it is correct.

In [13]:
def LogisticLossFun(w, X, y, lam):
    # Computes the cost function for all the training samples
    m = X.shape[0]
    Xw = X.dot(w)
    yT = y.reshape(-1,1)
    yXw = np.multiply(yT,Xw)    
    f = np.sum(np.logaddexp(0,-yXw)) + 0.5*lam*np.sum(w*w)
    return f


def numericalGrad(funObj, w,epsilon):
    m = len(w)
    grad = np.zeros(m)
    for i in range(m):
        wp = np.copy(w)
        wn = np.copy(w)
        wp[i] = w[i] + epsilon
        wn[i] = w[i] - epsilon
        grad[i] = (funObj(wp) - funObj(wn))/(2*epsilon)
    return grad

In [14]:
n = 100
m = 10

X = np.random.rand(n,m)
y = np.random.rand(n)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

funObj = lambda w: LogisticLossFun(w,X,y,1)
[f,g] = LogisticLoss(w,X,y,1)
gn = numericalGrad(funObj, w, 1e-10)
fn = funObj(w)
print(f)
print(fn)
print(gn)
print(g)

123.24355436930067
123.24355436930067
[20.91184115 21.99328719 21.94660453 18.89297607 19.07018543 21.4787832
 18.11095274 20.62350291 20.89137752 19.96859567]
[[20.91181858]
 [21.99332411]
 [21.94641886]
 [18.89291572]
 [19.07008602]
 [21.47886715]
 [18.1108865 ]
 [20.62333919]
 [20.89130566]
 [19.96848919]]
